In [ ]:
!pip install -q numpy pandas matplotlib transformers PyPDF2 langchain-community \
                 faiss-cpu tiktoken pymupdf sentence-transformers langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 5.1 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
import fitz
import os

# Directory where your PDFs are stored
pdf_directory = '/content/'

# Get all PDF files in the directory
pdf_files = [f for f in os.listdir(pdf_directory) if f.lower().endswith('.pdf')]

# Process each PDF
for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_directory, pdf_file)
    print(f"\nProcessing: {pdf_file}\n{'-'*40}")

    # Open and extract text
    doc = fitz.open(pdf_path)
    full_text = "\n".join([page.get_text() for page in doc])
    doc.close()

    # Print the first 500 characters as preview
    print(full_text[:500] + ("..." if len(full_text) > 500 else ""))
    print(f"\nExtracted {len(full_text)} characters from {pdf_file}")

print(f"\nDone! Processed {len(pdf_files)} PDF files")


Processing: Bangladesh Labour Rules.pdf
----------------------------------------
 
✅ Bangladesh Labour Rules, 2015 
 
Introduction 
The Bangladesh Labour Rules, 2015 were formulated to give practical effect to the Bangladesh 
Labour Act, 2006, and to ensure clarity in procedures, compliance, and enforcement. These rules 
define and expand the administrative procedures for employment conditions, safety, welfare, 
working hours, holidays, wages, disciplinary procedures, and industrial relations. The rules aim 
to balance employer-employee relationships and protect workers from...

Extracted 5633 characters from Bangladesh Labour Rules.pdf

Processing: Traffic Law in Bangladesh.pdf
----------------------------------------
Traffic Law in Bangladesh  
Traffic laws in Bangladesh are primarily governed by the Road Transport Act 2018, which 
replaced the Motor Vehicles Ordinance 1983, and are enforced by the Bangladesh Road 
Transport Authority (BRTA). These laws aim to ensure road safety, re

In [ ]:
import fitz
import os
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. Setup
pdf_directory = '/content/'
encoder = tiktoken.encoding_for_model("gpt-4")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=lambda x: len(encoder.encode(x)),
)

# 2. Process PDFs
pdf_files = [f for f in os.listdir(pdf_directory) if f.lower().endswith('.pdf')]
all_chunks = []

for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_directory, pdf_file)

    # Extract text
    doc = fitz.open(pdf_path)
    full_text = "\n".join([page.get_text() for page in doc])
    doc.close()

    # Create chunks WITHIN the loop
    chunks = text_splitter.create_documents(
        [full_text],
        metadatas=[{"source": pdf_path}]  # or pdf_file for just the name
    )
    all_chunks.extend(chunks)

    print(f"Created {len(chunks)} chunks from {pdf_file}")

# 3. Final output
print(f"\nTotal: {len(all_chunks)} chunks from {len(pdf_files)} PDFs")

Created 2 chunks from Bangladesh Labour Rules.pdf
Created 2 chunks from Traffic Law in Bangladesh.pdf
Created 2 chunks from Drugs Law in Bangladesh.pdf
Created 2 chunks from Constitutional Law in Bangladesh.pdf
Created 3 chunks from Anti-Terrorism Act.pdf
Created 10 chunks from Legal Cases in Bangladesh.pdf
Created 2 chunks from Digital Security Act of Bangladesh.pdf
Created 21 chunks from Digital-Security-Act-2020.pdf
Created 2 chunks from Dowry Act of Bangladesh.pdf
Created 2 chunks from Child Marriage Act-2017.pdf
Created 7 chunks from Legal FAQ’s For Bangladesh.pdf
Created 2 chunks from Child and Women Protection Laws in Bangladesh.pdf
Created 2 chunks from Education and Environment Laws in Bangladesh.pdf
Created 2 chunks from Criminal Law in Bangladesh.pdf
Created 2 chunks from Civil Law in Bangladesh.pdf
Created 2 chunks from Cyber and ICT Law in Bangladesh.pdf

Total: 65 chunks from 16 PDFs


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': False}
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

# 4. Create FAISS Vector Store
db = FAISS.from_documents(
    documents=all_chunks,
    embedding=embedding_model
)

# 5. Save and Create Retriever
db.save_local("faiss_db")
print("FAISS index saved successfully")

FAISS index saved successfully


In [ ]:
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)

In [ ]:
# 6. Query the Retriever
query = "What is Digital Security Act?"
results = retriever.invoke(query)
print(results)

# Print results
for i, doc in enumerate(results, 1):
    print(f"\nResult {i}:")
    print(doc.page_content)
    print(f"Source: {doc.metadata.get('source')}")

[Document(id='b93a23c9-78e0-496a-a903-ec4e1adcabd6', metadata={'source': '/content/Digital-Security-Act-2020.pdf'}, page_content='†iwR÷vW© bs wW G-1 \n evsjv‡`k \n†M‡RU \nAwZwi³ msL¨v \nKZ…©cÿ KZ…©K cÖKvwkZ \n \niweevi, A‡±vei 6, 2019 \n \n \nGovernment of the People’s Republic of Bangladesh \nLegislative and Parliamentary Affairs Division \nMinistry of Law, Justice and Parliamentary Affairs \nNOTIFICATION \nDated:  30 September, 2019 AD/15 Ashwin, 1426 BE \n    S.R.O. NO. 310-Law/2019.\uf0beIn exercise of the powers conferred by \nsection 62 of the Digital Security Act, 2018, the Government is pleased to \npublish the following English translation of the Act to be called the Authentic \nEnglish Text of the Act, and it shall be deemed to have been effective from the \ndate on which the Act comes into force under sub-section (2) of section 1 of the \nAct: \nDIGITAL SECURITY ACT, 2018 \nAct No. XLVI of 2018  \nAn Act to make provisions for ensuring digital security and identification, \n

In [ ]:
from groq import Groq
from langchain.llms.base import LLM
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from typing import Any

In [ ]:
# initialize groq client
client = Groq(api_key="GROQ_API_KEY")

In [ ]:
class GroqLLM(LLM):
  groq_client: Any = None

  def __init__(self, groq_client: Any, **kwargs: Any):
    super().__init__(**kwargs)
    if groq_client:
      self.groq_client = groq_client

  def _call(self, prompt, stop = None):
    response = self.groq_client.chat.completions.create(
        messages = [{"role": "user", "content": prompt}],
        model = "llama-3.1-8b-instant",
        max_tokens = 256,
        temperature = 0.5
    )
    return response.choices[0].message.content

  @property
  def _llm_type(self):
    return "groq"

In [ ]:
llm = GroqLLM(client)

In [ ]:
from langchain_core.prompts import PromptTemplate

CUSTOM_PROMPT = """You are a legal expert specializing in every laws in Bangladesh. Answer the question using ONLY the provided context. If unsure, say "I cannot find a definitive answer."

Follow these rules:
1. Be precise and cite relevant sections when possible
2. Use bullet points for multi-part answers
3. Keep responses under 150 words
4. Never invent information

Context: {context}
Question: {question}

Answer in this format:
**Summary**: [1-2 sentence overview]
**Details**:
- [Key point 1 with section reference if available]
- [Key point 2]
**Source**: [document name]"""

prompt = PromptTemplate(
    input_variables = ["context", "question"],
    template = CUSTOM_PROMPT
)

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = FAISS.load_local("faiss_db", embedding_model, allow_dangerous_deserialization=True)

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")

/tmp/ipython-input-3634644841.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")


In [ ]:
chain = ConversationalRetrievalChain.from_llm(
    llm = llm,
    retriever = retriever,
    memory = memory,
    return_source_documents = True,
    combine_docs_chain_kwargs = {"prompt": prompt}
)
print("Conversational Retrieval Chain loaded successfully")

Conversational Retrieval Chain loaded successfully


In [ ]:
test_queries = [
    "What is Digital Security Act?",
    "What are penalties mentioned in the act?",
    "How to file a domestic violence complaint?"
]

for query in test_queries:
    result = chain({"question": query})
    print(result['answer'])

/tmp/ipython-input-554648193.py:8: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = chain({"question": query})


**Summary**: The Digital Security Act, 2018 is an Act to ensure digital security and identification, prevent, suppress, and trial of offences committed through digital devices.

**Details**:
- It is an Act to make provisions for ensuring digital security and identification, prevention, suppression, and trial of offences committed through digital devices and for matters ancillary thereto. (Section 1)
- The Act shall come into force at once. (Section 1)
- The Act establishes an Agency to be called the Digital Security Agency consisting of 1 (one) Director General and 2 (two) Directors. (Section 5)
- The Act empowers the Government to make rules for carrying out the purposes of this Act, including establishment of digital forensic lab, supervision of digital forensic lab, and security of critical information infrastructure. (Section 60)

**Source**: The Digital Security Act, 2018 (Act No. XLVI of 2018)
**Summary**: The Digital Security Act, 2018, prescribes various penalties for offenses 

In [ ]:
def summarize_pdf(pdf_file):
    """Extract text from PDF and create summary (using 1st code's logic with 2nd code's variables)"""
    # Extract text from PDF (using same method as your existing code)
    doc = fitz.open(pdf_file.name)
    full_text = "\n".join([page.get_text() for page in doc])
    doc.close()

    if not full_text.strip():
        return "Error: No text could be extracted from the PDF. It might be empty, scanned, or protected."

    print(f"Extracted text from {pdf_file.name} ({len(full_text)} characters)")

    encoded = encoder.encode(full_text)
    max_tokens = 6000

    if len(encoded) > max_tokens:
        print(f"Document is large ({len(encoded)} tokens). Processing in chunks...")

        # Split into chunks for summarization
        chunks = text_splitter.create_documents([full_text], metadatas=[{"source": pdf_file.name}])

        # Summarize each chunk
        chunk_summaries = []
        for i, chunk in enumerate(chunks[:10]):  # Limit to first 10 chunks to avoid excessive processing
            chunk_prompt = f"""Summarize the following text section concisely, focusing on key points:

{chunk.page_content}

Provide a brief summary (2-3 sentences):"""

            summary = llm._call(chunk_prompt)
            chunk_summaries.append(summary)

            if (i + 1) % 3 == 0:
                print(f"Processed {i + 1} chunks...")

        # Combine chunk summaries
        combined = "\n\n".join(chunk_summaries)

        final_prompt = f"""Create a comprehensive summary of this document based on the following section summaries:

{combined}

Provide a well-structured summary covering:
1. Main topic and purpose
2. Key points and findings
3. Important conclusions or recommendations

Summary:"""

    else:
        final_prompt = f"""Please provide a comprehensive summary of the following document:

{full_text}

Create a well-structured summary covering:
1. Main topic and purpose
2. Key points and findings
3. Important conclusions or recommendations

Summary:"""

    # Generate final summary
    final_summary = llm._call(final_prompt)
    return final_summary

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Specify your case PDF file name here
CASE_PDF_NAME = "Legal Cases in Bangladesh.pdf"

# Extract only case texts from your specific case PDF
case_texts = []
case_sources = []

for chunk in all_chunks:
    # Check if this chunk is from the case PDF
    if CASE_PDF_NAME in chunk.metadata['source']:
        case_texts.append(chunk.page_content)
        case_sources.append(chunk.metadata['source'])

print(f"Loaded {len(case_texts)} case chunks from {CASE_PDF_NAME} for similarity search")

# Create TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words='english',
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.8
)

# Fit and transform the case texts
tfidf_matrix = tfidf_vectorizer.fit_transform(case_texts)
print("TF-IDF matrix created successfully")

# Function to find similar cases with FULL TEXT
def find_similar_cases(query_text, top_k=5):
    # Transform query using the same vectorizer
    query_tfidf = tfidf_vectorizer.transform([query_text])

    # Calculate cosine similarity
    similarity_scores = cosine_similarity(query_tfidf, tfidf_matrix).flatten()

    # Get top similar cases
    top_indices = similarity_scores.argsort()[-top_k:][::-1]

    similar_cases = []
    for idx in top_indices:
        similar_cases.append({
            'text': case_texts[idx],
            'source': case_sources[idx],
            'similarity_score': similarity_scores[idx]
        })

    return similar_cases

# Test the case similarity search
test_case_queries = [
    "domestic violence case legal precedent",
    "property dispute inheritance law",
    "contract breach commercial law"
]

for query in test_case_queries:
    print(f"\nQuery: {query}")
    print("-" * 50)

    similar_cases = find_similar_cases(query, top_k=3)

    for i, case in enumerate(similar_cases, 1):
        print(f"Text Preview: {case['text']}")
        print("-" * 30)

print("\nCase Similarity Search feature loaded successfully!")

Loaded 10 case chunks from Legal Cases in Bangladesh.pdf for similarity search
TF-IDF matrix created successfully

Query: domestic violence case legal precedent
--------------------------------------------------
Text Preview: 6. Labor and Employment Law Cases 
 
Case 26: Bangladesh Garment Workers Union vs. Export Garments Ltd. 
(2010) 
Court: Labor Court, Dhaka 
Case Number: Labor Case 123/2009 
Date: May 18, 2010 
Facts: Garment workers sought compensation for workplace accident, improved 
safety measures, and recognition of union rights. 
Legal Issues: Workplace safety, industrial accidents, workers' compensation, 
union rights, employer liability. 
Judgment: Compensation awarded to injured workers and directions issued for 
safety improvements. 
Significance: Strengthened workplace safety standards and workers' rights in 
garment industry. 
 
Case 27: Dr. Khaleda Rahman vs. Dhaka University (2013) 
Court: High Court DivisionRetryMMContinueEditCase 27: Dr. Khaleda Rahman 
vs. Dhaka 

In [ ]:
import gradio as gr

# Legal Chatbot
def legal_chatbot(message, history):
    result = chain({"question": message})
    # Add the new message and response to history
    history.append([message, result['answer']])
    return history, ""

# PDF Summarizer
def pdf_summarizer(pdf_file):
    if pdf_file is None:
        return "Please upload a PDF file."

    try:
        summary = summarize_pdf(pdf_file)
        return summary
    except Exception as e:
        return f"Error processing PDF: {str(e)}"

# Case Similarity Search function
def case_search(query_text):
    if not query_text.strip():
        return "Please enter a search query."

    similar_cases = find_similar_cases(query_text, top_k=3)

    if not similar_cases:
        return "No similar cases found."

    result = f"**Similar Cases for: '{query_text}'**\n\n"

    for i, case in enumerate(similar_cases, 1):
        result += f"**Case {i}:**\n"
        result += f"{case['text']}\n"
        result += "-" * 80 + "\n\n"

    return result

# Create Gradio interface
with gr.Blocks(title="AI Legal Aid System for Bangladesh", theme=gr.themes.Soft()) as demo:

    gr.Markdown("# 🏛️ AI-Powered Legal Aid for Bangladesh")
    gr.Markdown("Get legal assistance, document summaries, and find similar cases using AI technology.")

    with gr.Tabs():

        # Tab 1: Legal Chatbot
        with gr.TabItem("💬 Legal Chatbot"):
            gr.Markdown("### Ask any legal question about Bangladesh law")

            chatbot = gr.Chatbot(
                height=400,
                placeholder="Legal AI Assistant will appear here..."
            )

            msg = gr.Textbox(
                placeholder="Type your legal question here... (e.g., 'What is Digital Security Act?')",
                label="Your Question"
            )

            clear = gr.Button("Clear Chat")

            msg.submit(legal_chatbot, [msg, chatbot], [chatbot, msg])
            clear.click(lambda: ([], ""), None, [chatbot, msg])

        # Tab 2: PDF Document Summarizer
        with gr.TabItem("📄 Document Summarizer"):
            gr.Markdown("### Upload a PDF to get an AI-generated summary")

            with gr.Row():
                with gr.Column():
                    pdf_input = gr.File(
                        label="Upload PDF Document",
                        file_types=[".pdf"],
                        type="filepath"
                    )

                    summarize_btn = gr.Button("Generate Summary", variant="primary")

                with gr.Column():
                    summary_output = gr.Textbox(
                        label="Document Summary",
                        lines=15,
                        placeholder="Upload a PDF and click 'Generate Summary' to see the AI-generated summary here..."
                    )

            summarize_btn.click(
                pdf_summarizer,
                inputs=pdf_input,
                outputs=summary_output
            )

        # Tab 3: Case Similarity Search
        with gr.TabItem("⚖️ Case Similarity Search"):
            gr.Markdown("### Find similar legal cases and precedents")

            with gr.Row():
                with gr.Column():
                    case_query = gr.Textbox(
                        label="Search Query",
                        placeholder="Enter legal terms or case description (e.g., 'domestic violence case')",
                        lines=3
                    )

                    search_btn = gr.Button("Search Similar Cases", variant="primary")

                with gr.Column():
                    case_results = gr.Textbox(
                        label="Similar Cases Found",
                        lines=15,
                        placeholder="Enter a search query and click 'Search Similar Cases' to find relevant legal precedents..."
                    )

            search_btn.click(
                case_search,
                inputs=case_query,
                outputs=case_results
            )

    # Footer
    gr.Markdown("---")
    gr.Markdown("**Note:** This AI system is designed to assist with legal information. Always consult with qualified legal professionals for official legal advice.")

# Launch the interface
demo.launch(debug=True, share=True)

/tmp/ipython-input-2263791858.py:52: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://20ce8e156f509ae64e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Extracted text from /tmp/gradio/47ce02272453e35aeb39d35850523d8ebfd9b303a5412c24e8dae8e3c86b66a5/Drugs Law in Bangladesh.pdf (5798 characters)
